In [9]:
import pandas as pd
import nibabel as nib
from os.path import join
import numpy as np

csv = 'PET_AUTOPSY.csv'

df = pd.read_csv(csv)

for index_, row_ in df.iterrows():

    alz_path = row_['ALZHEIMER PATHOLOGY']
    lb_path = row_['LEWY PATHOLOGY']

    if alz_path >= 2 and lb_path < 2:
        group = 'pure-AD'
    elif alz_path < 2 and lb_path >= 2:
        group = 'pure-LB'
    elif alz_path >= 2 and lb_path >= 2:
        group = 'Mixed'
    else:
        group = 'Negative'

    df.loc[index_, 'group'] = group

df = df.loc[df['group'] != 'Mixed']
df = df.loc[df['group'] != 'Negative']


In [10]:
atlas_path = '/Users/jsilva/repositories/MIA/Module_3 (english)/CDSS_Imaging/cortex_Harvard_Oxford_orig_atlas.nii.gz'
atlas_csv = '/Users/jsilva/repositories/MIA/Module_3 (english)/CDSS_Imaging/Harvard_Oxford.csv'

patient_path = '/Users/jsilva/data/MIA/Autopsy/'

df_atlas = pd.read_csv(atlas_csv)

img_atlas = nib.load(atlas_path)
data_atlas =img_atlas.get_fdata()

for index_, row_ in df.iterrows():

    subject_id = row_['SUBJECT_ID']
    image_id = row_['FDG_ID']
    img_path = join(patient_path, '%s_I%s' % (subject_id,image_id), 'swfdg_normhist.hdr')

    img_ = nib.load(img_path)
    data_ = img_.get_fdata()

    for index_atlas, row_atlas in df_atlas.iterrows():

        roi_name = row_atlas['ROI_NAME']
        roi_num = row_atlas['ROI_NUM']

        indx_roi_ = np.where(data_atlas == roi_num)
        mean_value = np.mean(data_[indx_roi_])

        df.loc[index_, roi_name] = mean_value



In [12]:
# Most significant differences.
from scipy.stats import ttest_ind
df_ad = df[df['group'] == 'pure-AD']
df_lb = df[df['group'] == 'pure-LB']

for roi in df.columns[9:]:

    t,p = ttest_ind(df_ad[roi], df_lb[roi])
    if p < 0.005:
        print(roi, round(p,3))
    else:
        df = df.drop(columns=[roi])

Inferior_Temporal_Gyrus_anterior_division 0.001
Intracalcarine_Cortex 0.002
Supracalcarine_Cortex 0.005


In [58]:
X = df.iloc[:,-3:]
y = df['group']
X

,Inferior_Temporal_Gyrus_anterior_division,Intracalcarine_Cortex,Supracalcarine_Cortex
0,1.082464,1.806612,1.867905
1,1.251840,1.562329,1.582445
4,1.169544,1.659618,1.659314
6,1.314990,1.516506,1.496108
9,1.059917,1.709761,1.648271
10,1.135093,1.804450,1.632121
13,1.075888,1.817859,1.800605
14,1.185621,1.575078,1.542219
16,1.133862,1.869121,1.797530
18,1.135471,1.708735,1.623819


In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import pickle


#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)

model = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=10.0, fit_intercept=True, intercept_scaling=1, class_weight="balanced", random_state=None, solver='lbfgs', max_iter=100)

model2 = SVC(C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight="balanced", verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None)

model.fit(X_train,y_train)
model2.fit(X,y)

y_pred = model2.predict(X)

conf_matrix = confusion_matrix(y, y_pred)
accuracy_score = accuracy_score(y, y_pred)
print("Accuracy = ", round(accuracy_score,2)*100, " %")
conf_matrix

pickle.dump(model, open('pretrained.pkl', 'wb'))


Accuracy =  89.0  %
